In [ ]:
from dataclasses import dataclass
from typing import List

import numpy as np

import pandas as pd


@dataclass
class Asset:
    """Product defined in Tab 1 (Asset Management)."""

    name: str
    distribution_type: str  # e.g., "normal"
    mean_return: float  # e.g., 0.08
    std_dev: float  # e.g., 0.15


@dataclass
class InvestmentScenario:
    """All parameters for one simulation run defined in Tab 2."""

    initial_capital: float
    horizon_years: int
    num_trials: int
    # List of (Asset, Weight) tuples for the portfolio composition
    portfolio_composition: List[tuple[Asset, float]]

In [2]:
def generate_annual_return(asset: Asset) -> float:
    """Draws a single random annual return based on the asset's distribution parameters."""

    # For now, only the normal distribution is implemented
    if asset.distribution_type == "normal":
        # Draw from the Normal Distribution N(mu, sigma)
        return np.random.normal(asset.mean_return, asset.std_dev)

    # Placeholder for future distributions
    # elif asset.distribution_type == "uniform":
    #     return np.random.uniform(asset.mean_return - asset.std_dev, asset.mean_return + asset.std_dev)

    else:
        # Fallback for undefined distribution type
        raise ValueError(f"Unsupported distribution type: {asset.distribution_type}")

In [11]:
def run_monte_carlo_simulation(scenario: Scenario) -> pd.DataFrame:
    """
    Executes the Monte Carlo simulation and returns a DataFrame in 'long' format.
    Each row represents a unique combination of (Trial, Year).
    """

    # Initialize a dictionary to efficiently store the results for all columns
    data = {
        "Trial": [],
        "Year": [],
        "Start_Value": [],
        "Total_Return": [],
        "End_Value": [],
    }

    # Pre-populate return columns for all assets
    asset_names = [asset.name for asset, _ in scenario.portfolio_composition]
    for name in asset_names:
        data[f"{name}_Return"] = []

    # Array to track the current portfolio value for each trial separately
    current_values = np.full(scenario.num_trials, scenario.initial_capital, dtype=float)

    for year in range(1, scenario.horizon_years + 1):
        # Calculate the random returns for ALL trials for this specific year
        # This is more efficient than the inner loop
        annual_returns_for_all_trials = []

        # 1. Calculate weighted annual return for each trial
        weighted_annual_returns = np.zeros(scenario.num_trials)

        # Loop through each unique asset in the scenario
        for asset, weight in scenario.portfolio_composition:
            # Generate N random returns (where N = num_trials) for THIS asset
            if asset.distribution_type == "normal":
                asset_returns = np.random.normal(
                    asset.mean_return, asset.std_dev, scenario.num_trials
                )
            elif asset.distribution_type == "uniform":
                # For cash, using a simple uniform draw:
                asset_returns = np.random.uniform(
                    asset.mean_return - asset.std_dev,
                    asset.mean_return + asset.std_dev,
                    scenario.num_trials,
                )
            else:
                raise ValueError(
                    f"Unsupported distribution type: {asset.distribution_type}"
                )

            # Add the specific asset returns for this year/trial combination
            data[f"{asset.name}_Return"].extend(asset_returns)

            # Add to the total weighted return for each trial
            weighted_annual_returns += asset_returns * weight

        # 2. Update the portfolio values

        # Record the start value *before* update
        data["Start_Value"].extend(current_values.tolist())

        # Calculate end value (This is the crucial reinvestment step)
        end_values = current_values * (1 + weighted_annual_returns)

        # Record returns and end values
        data["Total_Return"].extend(weighted_annual_returns.tolist())
        data["End_Value"].extend(end_values.tolist())
        data["Trial"].extend(range(scenario.num_trials))
        data["Year"].extend([year] * scenario.num_trials)

        # 3. Update the tracking array for the next year's calculation
        current_values = (
            end_values  # The end value becomes the start value for the next year
        )

    # Final assembly into the DataFrame
    results_df = pd.DataFrame(data)
    return results_df

In [ ]:
# --- 1. Define the Base Assets ---
# These simulate the records loaded from your Asset Management tab/table
asset_a = Asset(
    name="Index Fund (S&P 500)",
    distribution_type="normal",
    mean_return=0.08,
    std_dev=0.15,
)
asset_b = Asset(
    name="Aggressive Tech Stock",
    distribution_type="normal",
    mean_return=0.12,
    std_dev=0.35,
)
asset_c = Asset(
    name="Corporate Bonds", distribution_type="normal", mean_return=0.04, std_dev=0.06
)


In [ ]:
# --- 2. Define the Portfolio Composition (Asset, Weight) ---
# Weights MUST sum to 1.0 (100%)
portfolio_mix = [
    (asset_a, 0.40),  # 40% in Index Fund
    (asset_b, 0.30),  # 30% in Aggressive Tech
    (asset_c, 0.30),  # 30% in Corporate Bonds
]

In [ ]:
# --- 3. Create the Final Scenario Object ---
test_scenario = Scenario(
    initial_capital=100_000,
    horizon_years=20,
    num_trials=1_000,
    portfolio_composition=portfolio_mix,
)


In [ ]:
print("✅ Dummy Scenario Created Successfully!")
print(f"Initial Capital: ${test_scenario.initial_capital:,.0f}")
print(
    f"Horizon: {test_scenario.horizon_years} years, Trials: {test_scenario.num_trials}"
)
print(
    "Portfolio Weights:",
    [
        (asset.name, f"{weight * 100:.0f}%")
        for asset, weight in test_scenario.portfolio_composition
    ],
)

✅ Dummy Scenario Created Successfully!
Initial Capital: $100,000
Horizon: 20 years, Trials: 1000
Portfolio Weights: [('Index Fund (S&P 500)', '40%'), ('Aggressive Tech Stock', '30%'), ('Corporate Bonds', '30%')]


In [12]:
test = run_monte_carlo_simulation(test_scenario)

In [13]:
test

,Trial,Year,Start_Value,Total_Return,End_Value,Index Fund (S&P 500)_Return,Aggressive Tech Stock_Return,Corporate Bonds_Return
0,0,1,1.000000e+05,-0.111965,8.880346e+04,-0.130244,-0.225057,0.025498
1,1,1,1.000000e+05,0.153409,1.153409e+05,0.150336,0.290320,0.020597
2,2,1,1.000000e+05,0.028260,1.028260e+05,0.077838,0.002937,-0.012519
3,3,1,1.000000e+05,0.027637,1.027637e+05,0.058462,-0.057005,0.071180
4,4,1,1.000000e+05,0.295532,1.295532e+05,0.046698,0.841850,0.080992
...,...,...,...,...,...,...,...,...
19995,995,20,2.519522e+05,-0.046297,2.402876e+05,-0.055705,-0.084853,0.004803
19996,996,20,1.169626e+06,0.246922,1.458432e+06,0.176051,0.507743,0.080596
19997,997,20,5.147301e+05,0.163710,5.989967e+05,-0.009033,0.495247,0.062498
19998,998,20,2.444105e+05,-0.000104,2.443850e+05,0.301162,-0.383838,-0.018057
